In [1]:
import pandas as pd
import numpy as np
import math
import os
from test_class import Test
os.environ['PYTHONHASHSEED'] = str(50)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import matplotlib.pyplot as plt

In [2]:
rounds = 50
dataset = pd.read_csv('RTT_data.csv')
data_args = {
    'data seed': 11,
    'distance clients': [0, 2, 3, 6],
    'distance augments': [0.5, 0.5, 0.5, 0.5],
    'tolerance': 5,
    'exclude dtypes': 'object',
    'drop labels': ['GroundTruthRange[m]'],
    'target labels': ['GroundTruthRange[m]'],
    'test size 1': 0.2,
    'test size 2': 0.25,
    'normalize': False,
    'client num': None
}

Mt = (np.ones(rounds) * 5).astype('int32')
model_seed = 50
test = Test(dataset, data_args, Mt, model_seed)

In [3]:
test.split(scheme = 3, args = (3, 4))

In [4]:
# perform cross validation over different seeds
seeds = [1, 2, 3, 4, 5]
fedavg_scores = []
conf_scores = []
for s in seeds:
    test.data_args['data seed'] = s
    cv_data = test.cross_validation(7, 1, 1, 1000, args = (3, 4), context = [0, 2])
    s1 = cv_data['s1 score']
    fedavg_scores.append(s1[0])
    conf_scores.append(s1[1])

CV Pair 1
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 121.37509369505355

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 167.94088461412082

FedAvg Avg Test Error: 4.562976780476656
ConFeddi Avg Test Error: 1.523817743303557
FedAvg MSE: 38.66193771362305
ConFeddi MSE: 5.944940567016602

CV Pair 2
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 119.1091078068078

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 173.0195948821655

FedAvg Avg Test Error: 1.324011250982127
ConFeddi Avg Test Error: 1.6384441162416445
FedAvg MSE: 4.531250476837158
ConFeddi MSE: 5.894400596618652

CV Pair 3
FedAvg
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Round 35
Round 40
Round 45
Round 50
Time: 120.5839552068079

ConFeddi
Round 5
Round 10
Round 15
Round 20
Round 25
Round 30
Ro

In [5]:
print(f'FedAvg Average S1 Score: {np.array(fedavg_scores).mean():.3f}')

FedAvg Average S1 Score: 1.958


In [9]:
print(f'ConFeddi Average S1 Score: {np.array(conf_scores).mean():.3f}')

ConFeddi Average S1 Score: 1.710


In [7]:
print(f'FedAvg S1 Score Variance: {np.array(fedavg_scores).var():.3f}')

FedAvg S1 Score Variance: 0.029


In [8]:
print(f'ConFeddi S1 Score Variance: {np.array(conf_scores).var():.3f}')

ConFeddi S1 Score Variance: 0.016
